### Preamble

The problem this attempts to solve is that I like using Jupyter Notebooks - they are great for annotating data and distributing code that we're working on - but don't much like the JupyerLab/Hub interface for writing code. 

The basic idea here is to set up a new [Python](http://www.python.org) project repository on a desktop using a modern coding environment and then publish it to Github. The project can then be pulled from Github into JupyerLab/Hub.

This guide assumes you have a Github account and are working on some flavour of Linux as an operating system (you mileage may vary otherwise).

All of this set up can be done in the terminal of an [Intergrated Development Environment](https://www.simplilearn.com/tutorials/python-tutorial/python-ide) such as [Visual Studio Code](https://code.visualstudio.com/). A guide to setting up VScode is [here](./vscode-guide.md). 

The following setup uses [poetry](https://python-poetry.org/) to manage a virtual environment for the project and [pre-commit](https://pre-commit.com/) to apply coding formatters and linters to the code before it is committed to git - thereby giving the code a consistent style.


### General system-wide setup

You should not install additional Python packages into the system-wide Python installation, rather you should use a separate [virtual enviroment][vit-env] to manage the dependencies of each project. If a package needs to be system-wide (here both git and poetry) we can use [pipx](https://github.com/pypa/pipx).

[Install pipx][pipx-install]:
```bash
    $ sudo python3 -m pip install --user pipx
    $ sudo python3 -m pipx ensurepath
```
Install [git][git]:
```bash
    $ sudo dnf/apt install git
```
Install [poetry][poetry]:
```bash
    $ sudo pipx install poetry
```
Configure the following poetry defaults:
```bash
    $ poetry config virtualenvs.in-project true
    $ poetry config virtualenvs.prompt "(.venv)"
    $ poetry config virtualenvs.path " "
    $ poetry config --list
```
```python
    cache-dir = "///.cache/pypoetry"
    experimental.system-git-client = false
    installer.max-workers = null
    installer.modern-installation = true
    installer.no-binary = null
    installer.parallel = true
    keyring.enabled = true
    solver.lazy-wheel = true
    virtualenvs.create = true
    virtualenvs.in-project = true
    virtualenvs.options.always-copy = false
    virtualenvs.options.no-pip = false
    virtualenvs.options.no-setuptools = false
    virtualenvs.options.system-site-packages = false
    virtualenvs.path = " "
    virtualenvs.prefer-active-python = false
    virtualenvs.prompt = "(.venv)"
    warnings.export = true
```
[vit-env]: https://medium.com/pythoneers/why-do-we-need-a-virtual-environment-for-a-python-project-37a6af754044
[git]: https://git-scm.com/
[poetry]: https://python-poetry.org/
[pipx-install]: https://pipx.pypa.io/stable/installation/

### Setting up a new project

Initialise the project using poetry
```bash
    $ mkdir bioinformatics-development-setup
    $ cd bioinformatics-development-setup
    $ poetry init
```
Add the [black](https://github.com/psf/black) and [ruff](https://docs.astral.sh/ruff/) linters, and [pre-commit](https://pre-commit.com/) git hooks to the project
```bash
    $ poetry add black[jupyter] --group dev
    $ poetry add pre-commit --group dev
    $ poetry add ruff --group dev
    $ cat pyproject.toml
```
```toml
    [tool.poetry]
    name = "bioinformatics-development-setup"
    version = "0.1.0"
    description = "A basic development environment for bioinformatics"
    authors = ["Cymon J. Cox <cymon.cox@googlemail.com>"]
    license = "MIT"
    readme = "README.ipynb"

    [tool.poetry.dependencies]
    python = "^3.10"

    [tool.poetry.group.dev.dependencies]
    black = {extras = ["jupyter"], version = "^24.8.0"}
    pre-commit = "^3.8.0"
    ruff = "^0.6.5"

    [build-system]
    requires = ["poetry-core"]
    build-backend = "poetry.core.masonry.api"
```

##### Setup "pre-commit" hooks to linters

Write the following configuration file to the project directory:
```bash
    $ cat .pre-commit-config.yaml
```
```yaml
repos:
  - repo: https://github.com/astral-sh/ruff-pre-commit
    # Ruff version.
    rev: v0.6.5
    hooks:
      # Run the linter.
      - id: ruff
        args: [ --fix ]
      # Run the formatter.
      - id: ruff-format
```

Install the pre-commit configuration:
```bash
    $ pre-commit install
    pre-commit installed at .git/hooks/pre-commit
```

##### Initialise the project as a git repository

 Write the .gitignore file

```bash
    $ cat .gitignore
    # jupyter
    .ipynb_checkpoints/*

    # poetry
    .venv

    # VScode
    bioinformatics-development-setup.code-workspace
    .vscode

    #Others
    *.swp
    .ruff_cache
```

Initialise a git repository

```bash
    $ git init
    $ git status
    On branch main

    No commits yet
    Changes not staged for commit:
        (use "git add <file>..." to update what will be committed)
        (use "git restore <file>..." to discard changes in working directory)
            modified:   README.ipynb
```

Add the files to the repository and commit
```bash
    $ git add .
    $ git commit -m "Linting with ruff"
    ruff.....................................................................Passed
    ruff-format..............................................................Passed
    [main 4908f0a] Linting with ruff
    1 file changed, 10 insertions(+), 27 deletions(-)
```

##### Publishing the project to Github

You first must create the Github repository throught the Github web-interface
- Log in to [Github](https://github.com/) and open the Dashboard

- On the top-left above the Top Repositories listing is a green button `new`
- Give you new repository the same name that you gave the project directory; here `bioinformatics-development-setup`
- Choose a license, add a README.md, but do not add a default ```.gitignore```
- Create repository
- Copy the URL from the browser address bar:`https://github.com/cymon/bioinformatics-development-setup`

Back in your terminal in the project directory, add the URL as the git remote address:
```bash
    $ git remote add origin https://github.com/cymon/bioinformatics-development-setup
    $ git config --list
    user.email=cymon.cox@googlemail.com
    user.name=Cymon J. Cox
    pull.rebase=false
    init.defaultbranch=main
    core.repositoryformatversion=0
    core.filemode=true
    core.bare=false
    core.logallrefupdates=true
    remote.origin.url=https://github.com/cymon/bioinformatics-development-setup
    remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*
```
Then push to Github setting upstream and forcing the update to the remote repository - you should not use force `-f` again, and you could lose commits in Github if you do:
```bash
    $ git branch --set-upstream-to=origin/main main
    $ git push -u -f origin main
```
Thereafter use `$ git pull origin` `$ git push origin`

You can now pull you project from Github on to whatever machine you like...

```bash
    $ git clone https://github.com/cymon/bioinformatics-development-setup.git
    $ cd bioinformatics-development-setup
    $ poetry install
    The currently activated Python version 3.8.17 is not supported by the project (^3.10).
    Trying to find and use a compatible version. 
    Using python3.11 (3.11.9)
    Creating virtualenv bioinformatics-development-setup-fVyA5M0X-py3.11 in /home/cymon/.cache/pypoetry/virtualenvs
    Updating dependencies
    Resolving dependencies... (11.6s)

    Package operations: 14 installs, 0 updates, 0 removals

    - Installing distlib (0.3.8)
    - Installing filelock (3.16.0)
    - Installing platformdirs (4.3.3)
    - Installing cfgv (3.4.0)
    - Installing click (8.1.7)
    - Installing identify (2.6.1)
    - Installing mypy-extensions (1.0.0)
    - Installing nodeenv (1.9.1)
    - Installing packaging (24.1)
    - Installing pathspec (0.12.1)
    - Installing pyyaml (6.0.2)
    - Installing virtualenv (20.26.4)
    - Installing black (24.8.0)
    - Installing pre-commit (3.8.0)

    Writing lock file

    Installing the current project: bioinformatics-development-setup (0.1.0)

    $ source .venv/bin/activate
    $ cat .pre-commit-config.yaml 
    repos:
    - repo: https://github.com/astral-sh/ruff-pre-commit
        # Ruff version.
        rev: v0.6.5
        hooks:
        # Run the linter.
        - id: ruff
            args: [ --fix ]
        # Run the formatter.
        - id: ruff-format
    $ pre-commit install
    pre-commit installed at .git/hooks/pre-commit
```
and you are back up and running on a new machine in an identical development environment.
